In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
plt.rcParams.update({'font.size': 14})
import os

import re

In [2]:
EQUATIONS = [
    'wave', 
    'gas_dynamics', 
    'reaction_diffusion', 
    'kuramoto_sivashinsky', 
    'advection',
    'burgers',
]
EQUATION_REAL_NAMES = {
    'wave': 'Wave', 
    'gas_dynamics': 'Gas Dynamics', 
    'reaction_diffusion': 'Reaction-Diffusion', 
    'kuramoto_sivashinsky': 'Kuramoto-Sivashinsky', 
    'advection': 'Advection',
    'burgers': 'Burgers',

}
MODELS = [
    'feast', 
    'gat', 
    'gcn', 
    'point_gnn', 
    'point_transformer', 
    'kernelNN', 
    'graphpde', 
    'persistence',
    'cnn', 
    'resnet', 
    'neuralpde'
]
MODEL_REAL_NAMES = {
    'feast': 'FeaSt', 
    'gat': 'GAT', 
    'gcn': 'GCN', 
    'point_gnn': 'PointGNN', 
    'point_net': 'PointNet',
    'point_transformer': 'Point Transformer', 
    'kernelNN': 'KernelNN', 
    'graphpde': 'GraphPDE', 
    'persistence': 'Persistence',
    'cnn': 'CNN', 
    'resnet': 'ResNet', 
    'neuralpde': 'NeuralPDE'
}

TASK = "forecast"
SUPPORTS = ["cloud", "grid"]
NUM_POINTS = "high"
METRIC = [f"test_rollout_{i}" for i in range(1,17)]

listing = glob.glob("../output/csv_logs/*/high/*/*/*/metrics.csv")
rows = []
for file in listing:
    path = file.split("/")
    equation = path[5]
    model = path[6]
    csv = pd.read_csv(file)[[f"test_rollout_{i}" for i in range(1, 17)]].dropna()
    csv['equation'] = EQUATION_REAL_NAMES[equation]
    csv['model'] = MODEL_REAL_NAMES[model]
    csv = csv.set_index(['equation', 'model'])
    rows.append(csv)

results = pd.concat(rows)
results = results.query("model != \"PointNet\"")


In [3]:
results

test_rollout_1  test_rollout_2  \
equation             model                                               
Wave                 CNN                      0.001434        0.010996   
                     NeuralPDE                0.001699        0.010858   
                     ResNet                   0.001459        0.010329   
Gas Dynamics         CNN                      0.004204        0.017296   
                     NeuralPDE                0.003734        0.014553   
...                                                ...             ...   
Kuramoto-Sivashinsky GraphPDE                 0.007198        0.059355   
                     KernelNN                 0.006687        0.051397   
                     PointGNN                 0.006730        0.052549   
                     Persistence              0.142243        0.482181   
                     Point Transformer        0.004897        0.039136   

                                        test_rollout_3  test_rollout_4  \
equation             model                                               
Wave                 CNN                      0.035093        0.073743   
                     NeuralPDE                0.029375        0.052151   
                     ResNet                   0.032706        0.067625   
Gas Dynamics         CNN                      0.038840        0.069755   
                     NeuralPDE                0.030368        0.050412   
...                                                ...             ...   
Kuramoto-Sivashinsky GraphPDE                 0.193169        0.433975   
                     KernelNN                 0.158321        0.339433   
                     PointGNN                 0.159624        0.334471   
                     Persistence              0.871265        1.219627   
                     Point Transformer        0.127856        0.287423   

                                        test_rollout_5  test_rollout_6  \
equation             model                                               
Wave                 CNN                      0.120123        0.165976   
                     NeuralPDE                0.072729        0.088916   
                     ResNet                   0.102738        0.128124   
Gas Dynamics         CNN                      0.111115        0.163608   
                     NeuralPDE                0.074455        0.102024   
...                                                ...             ...   
Kuramoto-Sivashinsky GraphPDE                 0.794523        1.246442   
                     KernelNN                 0.597169        0.911650   
                     PointGNN                 0.575235        0.863840   
                     Persistence              1.494708        1.691459   
                     Point Transformer        0.521453        0.810471   

                                        test_rollout_7  test_rollout_8  \
equation             model                                               
Wave                 CNN                      0.207038        0.241552   
                     NeuralPDE                0.105090        0.124263   
                     ResNet                   0.146789        0.162218   
Gas Dynamics         CNN                      0.227226        0.301390   
                     NeuralPDE                0.132273        0.164706   
...                                                ...             ...   
Kuramoto-Sivashinsky GraphPDE                 1.720997        2.141097   
                     KernelNN                 1.242362        1.544169   
                     PointGNN                 1.168177        1.457609   
                     Persistence              1.817970        1.888432   
                     Point Transformer        1.114025        1.391032   

                                        test_rollout_9  test_rollout_10  \
equation             model                                                
Wave                 CNN              

In [5]:
results_single = results.pivot_table(values="test_rollout_16", index="model", columns="equation")
t = results_single.to_latex(float_format='%.2E')
t = results_single.to_markdown()
print(t)

| model             |       Advection |   Burgers |   Gas Dynamics |   Kuramoto-Sivashinsky |   Reaction-Diffusion |     Wave |
|:------------------|----------------:|----------:|---------------:|-----------------------:|---------------------:|---------:|
| CNN               |     0.00161331  |  0.554554 |       0.995382 |            1.26011     |          0.0183483   | 0.561433 |
| FeaSt             |     1.48288     |  0.561197 |       0.819594 |            3.74448     |          0.130149    | 1.61066  |
| GAT               | 41364.1         |  0.833353 |       1.21436  |            5.68925     |          3.85506     | 2.38418  |
| GCN               |     3.51453e+13 | 13.0876   |       7.20633  |            1.70612e+24 |          1.75955e+07 | 7.89253  |
| GraphPDE          |     1.07953     |  0.729879 |       0.969208 |            2.1044      |          0.0800235   | 1.02586  |
| KernelNN          |     0.897431    |  0.72716  |       0.854015 |            2.00334     |          0

/tmp/ipykernel_20315/4010742912.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  t = results_single.to_latex(float_format='%.2E')


In [6]:
scores = re.findall("\d\D\d\dE\D\d\d", t)
for s in scores:
    exponent = s[6:] if s[6] != "0" else s[7]
    sign = "" if s[5] == "+" else "-"
    formated_s = f"${s[:4]}\cdot 10^{{{sign}{exponent}}}$"
    t = t.replace(s, formated_s)

print(t)

| model             |       Advection |   Burgers |   Gas Dynamics |   Kuramoto-Sivashinsky |   Reaction-Diffusion |     Wave |
|:------------------|----------------:|----------:|---------------:|-----------------------:|---------------------:|---------:|
| CNN               |     0.00161331  |  0.554554 |       0.995382 |            1.26011     |          0.0183483   | 0.561433 |
| FeaSt             |     1.48288     |  0.561197 |       0.819594 |            3.74448     |          0.130149    | 1.61066  |
| GAT               | 41364.1         |  0.833353 |       1.21436  |            5.68925     |          3.85506     | 2.38418  |
| GCN               |     3.51453e+13 | 13.0876   |       7.20633  |            1.70612e+24 |          1.75955e+07 | 7.89253  |
| GraphPDE          |     1.07953     |  0.729879 |       0.969208 |            2.1044      |          0.0800235   | 1.02586  |
| KernelNN          |     0.897431    |  0.72716  |       0.854015 |            2.00334     |          0

In [7]:
UPPER_LIMIT_DICT = {
    'Wave': 3.1, 
    'Gas Dynamics': 2.1, 
    'Reaction-Diffusion': 4.1, 
    'Kuramoto-Sivashinsky': 4.1, 
    'Advection': 2.6,
    'Burgers': 2.1
}

for equation in EQUATIONS:
    equation = EQUATION_REAL_NAMES[equation]

    for model in MODELS:
        model_name = MODEL_REAL_NAMES[model]
        values = results.query(f"model == \"{model_name}\" and equation == \"{equation}\"").values.flatten()

        if model_name in ["CNN", "ResNet", "NeuralPDE"]:
            linestyle = "dotted"
        else:
            linestyle = "solid"

        if model_name == "Persistence":
            color = "black"
            linewidth = 3
            linestyle = "dashdot"
        else:
            color = None
            linewidth = 2.5
        
        #print(equation, model)
        #print(values)
        plt.plot(range(1, 17), values, label=model_name, linestyle=linestyle, linewidth=linewidth, c=color)
    

    plt.legend()
    plt.ylim(-0.1, UPPER_LIMIT_DICT[equation])
    plt.ylabel("MSE")
    plt.xlabel("Rollout step")
    plt.tight_layout()
    plt.savefig(f"../figures/results/{equation}.svg", dpi=200, bbox_inches="tight")
    plt.clf()

<Figure size 640x480 with 0 Axes>